<a href="https://colab.research.google.com/github/hane-momiji/Hello-World/blob/master/IOhw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
# sys.path.insert(0, '/root/userspace/IOhw2/materials')
import math

import numpy as np
# import tensorflow as tf
import pandas as pd


from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

random_seed = 34

np.random.seed(random_seed)
# tf.set_random_seed(random_seed)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# csv file's authorization code is 1M5Bmmlz8zxKpTbzdN8LJguCLpQaFDg5_
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.8MB/s 


In [4]:
downloaded = drive.CreateFile({'id':'1M5Bmmlz8zxKpTbzdN8LJguCLpQaFDg5_'})

# Download the file to a local disk as 'sample.csv'.
downloaded.GetContentFile('Data2019PS2.csv')

import pandas as pd
df = pd.read_csv('Data2019PS2.csv', header = None)
df.head()

,0,1,2
0,1.0,1.0,127379.0
1,1.0,2.0,128477.0
2,1.0,3.0,132527.0
3,1.0,4.0,136916.0
4,1.0,6.0,139476.0


In [5]:
# df =  pd.read_csv(filepath_or_buffer="/root/userspace/IOhw2/data/Data2019PS2.csv", encoding="ms932", sep=",")
data = df.values.astype(dtype = 'int32')
marketID = data[:,0]
productID = data[:,1]

char = np.arange(0.01,0.11, 0.01)
print("Characteristics vector is",char)

Characteristics vector is [0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 ]


In [0]:
# Set the numbers of Market size, markets, and observations.
MS = 1000000
nM = 1000
nm = data.shape[0]
ns = 1000 # num of simulations

In [0]:
from numpy.linalg import inv
from numpy.linalg import norm
# 説明変数を作る
x = np.zeros(shape = (nm,1), dtype = 'float32')
for i in range(10):
    idx = np.where(productID == i+1)[0]
#     print(i)
    # productIDがマッチする商品の特性を代入する
    x[idx] += char[i]
X = np.concatenate((np.ones(shape =(nm,1), dtype = 'float32'), x), axis = 1)

# print(x[0:10])

# outside option のシェアを計算する
def calc_outside(data, MS = 1000000):
    num = np.zeros(shape =(nM,1), dtype = 'float32')
    s0 = np.zeros(nM)
    for m in range(nM):
        idx = np.where(marketID == m+1)[0]
        output_inmarket = data[idx,2] # marketID がmのデータのみ抽出
        s0[m] = MS - np.sum(output_inmarket) #アウトサイドオプションの購入数
        num[m] = np.shape(idx)[0] #マーケットmに入る企業数
    return [s0, num]

# マーケットサイズの変更をした場合のbetaを計算する
def get_params(data, MS):
    outside = calc_outside(data, MS)
    s_out = outside[0]
    num = outside[1].astype(dtype = 'int32')
#     print(type(num))
    for m in range(nM):
        if (m == 0):
            # outside[0]がoutside optionの数, outside[1]はそのマーケットの商品数
            s0 = s_out[m]* np.ones(shape = (num[m]), dtype = 'float32' )
        else:
            # 初回以降は単純に継ぎ足ししていく。
            s0 = np.concatenate((s0, s_out[m]  * np.ones(shape = (num[m]), dtype = 'float32' )),  axis = 0)
    sJ = data[:, 2]/MS
    delta = np.log(sJ) - np.log(s0)
    beta = np.matmul( inv(np.matmul(np.transpose(X),X)),\
                     np.matmul(np.transpose(X), delta) )
    return [beta, delta, num, sJ]

# problem(a)
beta0 = get_params(data, MS = MS)[0]
beta1 = get_params(data, MS = 2000000)[0]
beta2 = get_params(data, MS = 4000000)[0]


In [8]:
print(beta0, beta1, beta2)

[-12.81724261   1.97382589] [-16.6783381    1.85913353] [-18.4410433    1.85525253]


In [0]:
# X = tf.placeholder(dtype = tf.float32, shape = [None,2])
# obs = tf.shape(X)[0]
# const = tf.Variable(tf.zeros(obs,1), dtype = tf.float32)


### BLP algorithm
まずはじめに、`randn`を使って乱数を生成する。  
そのあとで、deltaの初期値として、$\log(sJ) - \log(s0)$を`get_params`から
取り出す。

In [9]:
# generate random numbers
nu = np.random.randn(ns,1)
print(nu.shape)
# delta initial value
beta, delta_init, outside_num, sJ = get_params(data, MS = MS)
delta_init = delta_init[:, None]

(1000, 1)


outerloopを回して目的関数最小化問題を解きたいので、`innerloop`関数と
`outerloop`関数を定義する。  
まずは、`innerloop`から。

In [0]:
def calc_matmu(Sigma):
#   print(Sigma.shape)
#   print(nu.shape)
  x_2 = X[:,1]
  return np.matmul(x_2[:,None],  np.transpose(nu)) * Sigma[0,0]

def innerloop_subroutine_oldver(delta, sigma, outside_num):
  delta = delta[:,None]
  matdelta = np.matmul(delta, np.ones(shape = (1, ns)))
  Sigma = np.array([[sigma]])
  matmu = calc_matmu(Sigma)
  matexp = np.exp(matdelta + matmu)
  print(matexp.shape)
  # divide this matrix into 1000 matrix for each market
  labels = np.cumsum(outside_num)
  print(labels[0:3])
  for m in range(nM):
    if (m == 0):
      start = 0
      markexp = matexp[start:labels[m], :]
#       print(markexp[:,1])
      denom_m = np.sum(markexp, axis = 0) + 1
      Mexp = np.matmul(markexp, inv(np.diag(denom_m)))
#       print(Mexp.shape)
      start = labels[m]
    else:
      markexp = matexp[start:(labels[m]), :]
      denom_m = np.sum(markexp, axis = 0) + 1
      Mexp = np.concatenate((Mexp, np.matmul(markexp, inv(np.diag(denom_m)))), axis = 0)
      if (m % 100 == 0):
        print('MarketID is: {}'.format(m))
      start = labels[m]
  
  # calculate the profile of shares from delta
  sRC = np.sum(Mexp, axis = 1)/ns
  diff = np.log(sJ) - np.log(sRC)
  newdelta = delta + diff
  dist = norm(diff)
  return newdelta, dist

def innerloop_subroutine(delta, sigma, outside_num):
  delta = delta[:,None]
  matdelta = np.matmul(delta, np.ones(shape = (1, ns)))
  Sigma = np.array([[sigma]])
  matmu = calc_matmu(Sigma)
  matexp = np.exp(matdelta + matmu)
  print(matexp.shape)
  # divide this matrix into 1000 matrix for each market
  labels = np.cumsum(outside_num) # 何行目でマーケットが変わるか、という合図。
  print(labels[0:3])
  for m in range(nM):
    if (m == 0):
      start = 0
      nprod_m = labels[m] # マーケットmでの商品の種類
      end = start + nprod_m # マーケットmで一番最後の商品の行の番号
      markexp = matexp[start:end , :] #マーケットmに限定した小行列
#       print(markexp[:,1])
      denom_m = np.sum(markexp, axis = 0) + 1
      denom_mat = np.matlib.repmat(denom_m, (npord_m,1))
#       print(Mexp.shape)
    else:
      start = labels[m-1]
      end = labels[m]
      nprod_m = end - start
      markexp = matexp[start:end, :]
      denom_m = np.sum(markexp, axis = 0) + 1
      denom_mat = np.concatenate((denom_mat, denom_m), axis = 0)
      if (m % 100 == 0):
        print('MarketID is: {}'.format(m))
  
  print(denom_mat.shape)
  # calculate the profile of shares from delta
  Mexp = matexp * denom_mat
  sRC = np.sum(Mexp, axis = 1)/ns
  diff = np.log(sJ) - np.log(sRC)
  newdelta = delta + diff
  dist = norm(diff)
  return newdelta, dist

def nevoshort(delta):
  N = np.shape(X)[0]
  if (np.shape(delta)[0] != N):
    print('the length does not match.')
    return 0
  beta = np.matmul(inv(np.matmul(np.transpose(X), X)),  np.matmul(np.transpose(X), delta))
  xi = delta - np.matmul(X, beta)
  return xi

def outerloop_subroutine(sigma):
  sigma = norm(sigma)
  dist = 10
  delta = delta_init
  i = 0
  while (dist > 1e-6):
    newdelta, dist = innerloop_subroutine(delta, sigma, outside_num)
    delta = newdelta
    i += 1
    if ((i +1) % 10 == 0):
      print('Innerloop Iteration: {}'.format(i))
  # ここでdeltaを得たので、xiと目的関数の計算に移る
  xi = nevoshort(delta)
  M = np.shape(xi)[0]
  W = np.eye(M)
  Q = np.matmul(np.matmul(np.transpose(xi), W), xi)
  return Q

  
    

In [0]:
sigma_init = 0.5

innerloop_subroutine(delta_init, sigma_init, outside_num)

## いよいよOuterloop


In [0]:
from scipy.optimize import fmin
import matplotlib.pyplot as plt

# fmin(outerloop_subroutine,[[sigma_init,]])

count = 0
plt.axis([0, 100, 0, 6.5])
plt.ion()

def cbf(sigma):
    global count
    count += 1
    f = outerloop_subroutine(sigma)
#     print('%d, %f,  %f' % (count, sigma, f))
    plt.scatter(count, f)

# banana = lambda X, a: 100*(X[1] - X[0]**2)**2 + (a - X[0])**2
# a = math.sqrt(2)
# arg = (a, )
[xopt, fopt, iter, funcalls, warnflag, allvecs] = fmin(outerloop_subroutine,  \
                                                        sigma_init, \
                                                        xtol=1e-4, ftol=1e-4,\
                                                       maxiter=400, maxfun=400,\
                                                      retall = True, full_output = True)
# 動かしたくない変数があるときはarg = argを引数に追加。